In [7]:
from scripts.tools.issuetracker import *
import os
import shutil
from officelib import wordlib
import dateutil.parser
import dateutil.tz
import datetime

In [189]:

def iter_gantt_outline(gantt):
    seen = set()
    level = 0
    for i in gantt:
        seen.clear()
        p = i
        level = 0
        while True:
            if p in seen:
                raise ValueError("Circular graph detected: %s" % seen)
            seen.add(p)
            p = p.parent
            if p is None:
                break
            level += 1
        yield level, i
                
            
def make_tree(gantt):
    tree = {}

    for iss in gantt:
        path = [iss]
        p = iss.parent
        while p is not None:
            path.append(p)
            p = p.parent
        v = tree
        while path:
            i = path.pop()
            if i not in v:
                v[i] = {}
            v = v[i]
    return tree


def make_tree2(gantt):
    tree = {}
    seen = set()
    for iss in gantt:
        path = [iss]
        if iss in seen:
            continue
        seen.add(iss)
        p = iss.parent
        while p is not None:
            path.append(p)
            seen.add(p)
            p = p.parent
        v = tree
        while path:
            i = path.pop()
            if i not in v:
                v[i] = {}
            v = v[i]
    return tree


def append_line(lines, lvl, issue, domain):
    link = "https://" + domain + "/issues/%d" % issue.id
    lines.append((lvl, "Issue #%d: %s" % (issue.id, issue.subject), link, issue))


def parse_tree(tree, lines, domain, level, set_gantt, unmodified):
    for k in sorted(tree, key=lambda iss: iss.id):
        if k not in set_gantt:
            unmodified.add(k)
        append_line(lines, level, k, domain)
        parse_tree(tree[k], lines, domain, level+1, set_gantt, unmodified)
            

def create_outline(gantt, domain, set_gantt):
    lines = []
    tree = make_tree2(gantt)
    unmodified = set()
    parse_tree(tree, lines, domain, 0, set_gantt, unmodified)
    return lines, unmodified
    
    
# def create_outline(gantt, domain, set_gantt):
#     lines = []
    
#     def append(lvl, issue):
#         link = "https://" + domain + "/issues/%d" % issue.id
#         lines.append((lvl, "Issue #%d: %s" % (issue.id, issue.subject), link))
        
#     for lvl, issue in iter_gantt_outline(gantt):
#         p = issue.parent
#         plvl = lvl - 1
#         while p is not None and p not in set_gantt:
#             set_gantt.add(p)
#             append(plvl, p)
#             p = p.parent
#             plvl -= 1
#         append(lvl, issue)
#     return lines

In [190]:
def make_style(doc, name="PY_FRS_TITLE", font_size=24, font="Calibri", 
                     alignment=None, indent=None, tabstops=None, space_after=None):
    
    if alignment is None:
        alignment = wordlib.c.wdAlignParagraphCenter
    
    style = doc.Styles.Add(name)
    style.BaseStyle = doc.Styles("Normal")
    style.NoSpaceBetweenParagraphsOfSameStyle = True
    
    style.Font.Size = font_size
    style.Font.Name = font
    
    style.ParagraphFormat.Alignment = alignment
    style.ParagraphFormat.SpaceBeforeAuto = False
    style.ParagraphFormat.SpaceAfterAuto = False
    style.ParagraphFormat.LineSpacingRule = wordlib.c.wdLineSpaceSingle
    
    if indent is not None:
        style.ParagraphFormat.LeftIndent = wordlib.inches_to_points(indent)
    
    if tabstops is not None:
        style.ParagraphFormat.TabStops.Add(wordlib.inches_to_points(tabstops), 
                                           wordlib.c.wdAlignTabLeft, 
                                           wordlib.c.wdTabLeaderDots)
    
    if space_after is not None:
        style.ParagraphFormat.SpaceAfter = space_after
    
    return style

In [191]:
def move(r, n):
    r.MoveStart(wordlib.c.wdCharacter, n)

def add_index_lines(doc, r, lines, unmodified):
    styles = {}
    nlines = len(lines)
    for i, (lvl, itxt, href, issue) in enumerate(lines, 1):
        
        print("\rCreating index %d of %d       " % (i, nlines), end="")
        
        style_name = "PY_FRS_BODY_%d" % lvl
        style = styles.get(style_name)
        if style is None:
            indent = lvl * 0.1
            style = make_style(doc, style_name, 12, "Calibri", 
                               wordlib.c.wdAlignParagraphLeft, indent, 6, 3)
            styles[style_name] = style
        
        r.Text = itxt
        r.Style = style
        
        if issue in unmodified:
            move(r, len(itxt))
            r.Text = " [unmodified]\t"
            r.Font.Size = 10
            move(r, len(" [unmodified]\t"))
        else:
            r.InsertAfter("\t")
            move(r, len(itxt)+1)
        
        doc.Hyperlinks.Add(r, href, "", href, "Link")
        r.InsertAfter("\r")
        move(r, 20)  # enough for "link"
        doc.Paragraphs.Add()
    print()

def create_word_index(lines, doc, index_title, from_date, to_date, unmodified):
    
    print("Creating Word Document...")
    
    pgs = doc.Paragraphs
    
    # title
    r = pgs(1).Range
    r.Text = index_title
    r.Style = make_style(doc, "PY_FRS_TITLE", 24, "Calibri", space_after=0)
    r.InsertAfter("\r")
    
    #subtitle
    move(r, len(index_title) + 2)
    ds1 = "%d/%d/%d" % (from_date.month, from_date.day, from_date.year)
    ds2 = "%d/%d/%d" % (to_date.month, to_date.day, to_date.year)
    txt = "Updated Specifications from %s to %s" % (ds1, ds2)
    r.Text = txt
    r.Style = make_style(doc, "PY_FRS_SUBTITLE", 12)
    r.InsertAfter("\r")
    
    # index
    move(r, len(txt) + 2)
    add_index_lines(doc, r, lines, unmodified)
    

def create_pdf_index(gantt, domain, index_title, from_date, to_date, set_gantt):
    print("Creating PDF Index...")
    lines, unmodified = create_outline(gantt, domain, set_gantt)
    word = wordlib.Word()
    doc = word.Documents.Add()
    with wordlib.lock_screen(word):
        create_word_index(lines, doc, index_title, from_date, to_date, unmodified)
    #create_word_index(lines, doc, index_title, from_date, to_date)
    

In [192]:
def should_include_issue(iss, from_date, to_date):
    return (iss.sprint_milestone.name == '3.0' and 
            iss.tracker.name == 'Specification' and
            from_date <= iss.updated_on <= to_date)
    
def parse_date(date):
    if isinstance(date, str):
        dt = dateutil.parser.parse(date)
        if dt.tzinfo is None:
            return datetime.datetime(dt.year, dt.month, dt.day, tzinfo=dateutil.tz.tzutc())
        return dt
    return date

def today():
    td = datetime.datetime.today()
    return datetime.datetime(td.year, td.month, td.day, tzinfo=dateutil.tz.tzutc())

In [193]:
def main(domain, user, password, from_date=None, to_date=None, index_title="FRS Update"):
    
    if from_date is None:
        from_date = datetime.datetime(1900, 1, 1, tzinfo=dateutil.tz.tzutc())
    else:
        from_date = parse_date(from_date)

    if to_date is None or to_date.lower() == "today":
        to_date = today()
    else:
        from_date = parse_date(to_date)
        
    api = IssuetrackerAPI(domain, user, password)
    gantt = api.download_issues('pbssoftware').values()
    gantt = [i for i in gantt if should_include_issue(i, from_date, to_date)]
    set_gantt = set(gantt)
    create_pdf_index(gantt, domain, index_title, from_date, to_date, set_gantt)
    #return gantt
    

In [194]:
word.ActiveDocument.Close(False)
main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', "2016-6-1", "today", 'Hello v3.0 FRS Update')

Creating PDF Index...
Creating Word Document...
Creating index 34 of 34       


In [17]:
# from scripts.tools.issuetracker import IssuetrackerAPI
# api= IssuetrackerAPI("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist')
# issues = api.download_issues('pbssoftware')
# gantt = issues.values()
# gantt2 = [i for i in gantt if should_include_issue(i, from_date, to_date)]

In [18]:
class IssuetrackerAPI():
    def __init__(self, *args):
        pass
    def download_gantt(self, *args):
        return gantt
    def download_issues(self, *args):
        return issues

In [16]:
from_date = parse_date("2016-6-1")
to_date = today()

In [141]:
g=iter_gantt_outline(gantt)
filtered = [i for i in gantt if should_include_issue(i, from_date, to_date)]
g2=iter_gantt_outline(filtered)
for i in gantt:
    if should_include_issue(i, from_date, to_date):
        pass
        #print(i.id)
x, y = list(zip(*g2))
iter_gantt_outline(filtered)

<generator object iter_gantt_outline at 0x000001C8A39049E8>

In [20]:
oldrepr = Issue.__repr__
newrepr = lambda self: "Issue(%d)" % self.id
Issue.__repr__ = Issue.__str__ = newrepr
_oldrepr = oldrepr

In [187]:
def make_tree2(gantt):
    tree = {}
    seen = set()
    for iss in gantt:
        path = [iss]
        if iss in seen:
            continue
        seen.add(iss)
        p = iss.parent
        while p is not None:
            path.append(p)
            seen.add(p)
            p = p.parent
        v = tree
        while path:
            i = path.pop()
            if i not in v:
                v[i] = {}
            v = v[i]
    return tree

In [188]:
p = []
p2 = []
domain = 'issue.pbsbiotech.com'
tree=make_tree(gantt2)
parse_tree(tree, p, domain, 0, set(), set())
tree2=make_tree2(gantt2)
parse_tree(tree2, p2, domain, 0, set(), set())
from itertools import zip_longest
class Dummy():
    id = ""
for (lvl, _, _, i1), (l2, _, _, i2) in zip_longest(p,p2, fillvalue = (0, "","",Dummy())):
    s1 = " " * lvl + str(lvl) + " "+ str(i1.id)
    s1 = "%-10s"%s1
    s2 = " " * lvl + str(lvl) +" "+ str(i2.id)
    s2 = "%-10s"%s2
    print(s1, s2, "%10s"% bool(i1==i2))
print(len(p), len(p2))

0 2712     0 2712           True
 1 989      1 989           True
 1 1845     1 1845          True
 1 1885     1 1885          True
 1 2725     1 2725          True
  2 1073     2 1073         True
  2 2908     2 2908         True
 1 2758     1 2758          True
 1 2808     1 2808          True
 1 2814     1 2814          True
 1 2825     1 2825          True
 1 2828     1 2828          True
 1 2851     1 2851          True
 1 3001     1 3001          True
  2 2819     2 2819         True
  2 2856     2 2856         True
  2 2859     2 2859         True
  2 2922     2 2922         True
0 2716     0 2716           True
 1 1583     1 1583          True
 1 2787     1 2787          True
0 2857     0 2857           True
 1 2880     1 2880          True
0 2876     0 2876           True
0 3165     0 3165           True
0 3176     0 3176           True
0 3178     0 3178           True
0 3194     0 3194           True
0 3223     0 3223           True
0 3225     0 3225           True
0 3226    

In [155]:
issues[2758].parent

Issue(2712)

In [183]:
def dump(d, level=0):
    for k in d:
        print(level * " ", k)
        dump(d[k], level+1)

dump(tree)
print("_________________")
dump(tree2)

 Issue(3194)
 Issue(3176)
 Issue(2716)
  Issue(1583)
  Issue(2787)
 Issue(3226)
 Issue(3240)
 Issue(3260)
 Issue(2712)
  Issue(2725)
   Issue(2908)
   Issue(1073)
  Issue(3001)
   Issue(2859)
   Issue(2922)
   Issue(2819)
   Issue(2856)
  Issue(2814)
  Issue(2808)
  Issue(1885)
  Issue(2828)
  Issue(1845)
  Issue(2758)
  Issue(2851)
  Issue(989)
  Issue(2825)
 Issue(3225)
 Issue(3230)
 Issue(3178)
 Issue(3165)
 Issue(2876)
 Issue(2857)
  Issue(2880)
 Issue(3223)
_________________
 Issue(3194)
 Issue(3176)
 Issue(2716)
  Issue(1583)
  Issue(2787)
 Issue(3001)
  Issue(2859)
  Issue(2922)
  Issue(2819)
  Issue(2856)
 Issue(3226)
 Issue(3240)
 Issue(3260)
 Issue(2712)
  Issue(1845)
  Issue(2814)
  Issue(2808)
  Issue(1885)
  Issue(2828)
  Issue(2758)
  Issue(2851)
  Issue(989)
  Issue(2825)
 Issue(3225)
 Issue(3230)
 Issue(2857)
  Issue(2880)
 Issue(3178)
 Issue(3165)
 Issue(2876)
 Issue(2725)
  Issue(2908)
  Issue(1073)
 Issue(3223)
